In [22]:
import numpy as np
from skimage import io, color
from time import time
import scipy as sc
import matplotlib.pyplot as plt
import sys
%matplotlib inline

In [23]:
sys.path.append('./lib/source/')

from errors import *
from data import *
from methods import *

In [24]:
def get_reduced_image(image, rank=10):
    U, S, V = np.linalg.svd(image)
    U = U[:, :rank]
    S = S[:rank]
    V = V[:rank, :]
    reduced_image = U.dot(np.diag(S).dot(V))
    reduced_image /= np.max(reduced_image)
    
    return reduced_image

In [25]:
def get_gray_image(image):
    gray_image = color.rgb2gray(image)
    gray_image /= np.max(gray_image)
    
    return gray_image

In [26]:
def save_image(image, name, folder_name):
    io.imsave('./data/images/' + folder_name + '/' + name + '.jpg', image)

In [34]:
def load_image(name, folder_name='original', max_size=1000):
    image = io.imread('./data/images/' + folder_name + '/' + name + '.jpg')
    image = image.astype('float')
    image /= 255.
    if image.ndim == 3:
        image = image[:max_size, :max_size, :]
    else:
        image = image[:max_size, :max_size]
            
    return image

In [42]:
for i in xrange(5):
    rank = 7
    image = load_image('pattern'+str(i+1))
    save_image(image, 'small_pattern'+str(i+1), 'original')
    gray_image = get_gray_image(image)
    reduced_image = get_reduced_image(gray_image, rank)
    save_image(reduced_image, str(rank)+'_pattern'+str(i+1), 'low_rank')
    M = reduced_image.copy()
    sample_fraction = 0.1
    dims = M.shape
    max_iter = int(1e3)
    tol = 1e-3
    nsamp = int(sample_fraction * np.prod(dims))

    omega = sampling_operator(nsamp, dims)
    vec_data = get_sampling_vector(M, omega)
    mat_data = get_sampling_matrix(M, omega)
    save_image(mat_data, str(i+1)+'_'+str(sample_fraction)+'_'+str(rank)+'_pattern', 'results')
    
    print 'Image: '+str(rank)+'_pattern'+str(i+1)+'. Size: %dx%d. Rank: %d. Sample fraction: %.2f.' % (M.shape[0], 
                                                                                                       M.shape[1], 
                                                                                                       np.linalg.matrix_rank(M),
                                                                                                       sample_fraction)

    t0 = time()
    X_svt = get_completion(omega, vec_data, rank, dims, 'SVT', max_iter, tol)
    t1 = time()
    rmse_svt = RMSE(X_svt, M)
    rank_svt = np.linalg.matrix_rank(X_svt)
    X_svt /= np.max(X_svt)
    save_image(X_svt, str(i+1)+'_'+str(sample_fraction)+'_'+str(rank)+'_pattern_SVT_RMSE_%.5f' % rmse_svt, 'results')
    print 'SVT. Rank: %d. RMSE: %.5f. Time: %d min %d sec.' % (rank_svt, rmse_svt, int(t1-t0) // 60, int(t1-t0) % 60)

    t0 = time()
    X_svp = get_completion(omega, vec_data, rank, dims, 'SVP', max_iter, tol)
    t1 = time()
    rmse_svp = RMSE(X_svp, M)
    rank_svp = np.linalg.matrix_rank(X_svp)
    X_svp /= np.max(X_svp)
    save_image(X_svp, str(i+1)+'_'+str(sample_fraction)+'_'+str(rank)+'_pattern_SVP_RMSE_%.5f' % rmse_svp, 'results')
    print 'SVP. Rank: %d. RMSE: %.5f. Time: %d min %d sec.' % (rank_svp, rmse_svp, int(t1-t0) // 60, int(t1-t0) % 60)

    t0 = time()
    X_si = get_completion(omega, vec_data, rank, dims, 'SoftImpute', int(max_iter), tol)
    t1 = time()
    rmse_si = RMSE(X_si, M)
    rank_si = np.linalg.matrix_rank(X_si)
    X_si /= np.max(X_si)
    save_image(X_si, str(i+1)+'_'+str(sample_fraction)+'_'+str(rank)+'_pattern_SI_RMSE_%.5f' % rmse_si, 'results')
    print ' SI. Rank: %d. RMSE: %.5f. Time: %d min %d sec.' % (rank_si, rmse_si, int(t1-t0) // 60, int(t1-t0) % 60)
    print ''

Image: 7_pattern1. Size: 300x500. Rank: 7. Sample fraction: 0.10.
SVT. Rank: 7. RMSE: 0.01162. Time: 1 min 24 sec.
SVP. Rank: 7. RMSE: 0.00262. Time: 0 min 3 sec.
 SI. Rank: 300. RMSE: 0.04935. Time: 0 min 4 sec.

Image: 7_pattern2. Size: 1000x820. Rank: 7. Sample fraction: 0.10.
SVT. Rank: 7. RMSE: 0.00311. Time: 2 min 42 sec.
SVP. Rank: 7. RMSE: 0.00222. Time: 0 min 11 sec.
 SI. Rank: 820. RMSE: 0.03745. Time: 0 min 23 sec.

Image: 7_pattern3. Size: 1000x1000. Rank: 7. Sample fraction: 0.10.
SVT. Rank: 7. RMSE: 0.00106. Time: 1 min 50 sec.
SVP. Rank: 7. RMSE: 0.00124. Time: 0 min 8 sec.
 SI. Rank: 1000. RMSE: 0.03898. Time: 0 min 26 sec.

Image: 7_pattern4. Size: 1000x1000. Rank: 7. Sample fraction: 0.10.
SVT. Rank: 7. RMSE: 0.00076. Time: 2 min 15 sec.
SVP. Rank: 7. RMSE: 0.00084. Time: 0 min 8 sec.
 SI. Rank: 1000. RMSE: 0.02533. Time: 0 min 26 sec.

Image: 7_pattern5. Size: 600x600. Rank: 7. Sample fraction: 0.10.
SVT. Rank: 7. RMSE: 0.00134. Time: 0 min 56 sec.
SVP. Rank: 7. RMSE

In [46]:
for i in xrange(5):
    rank = 11
    image = load_image('pattern'+str(i+1))
    save_image(image, 'small_pattern'+str(i+1), 'original')
    gray_image = get_gray_image(image)
    reduced_image = get_reduced_image(gray_image, rank)
    save_image(reduced_image, str(rank)+'_pattern'+str(i+1), 'low_rank')
    M = reduced_image.copy()
    sample_fraction = 0.1
    dims = M.shape
    max_iter = int(1e3)
    tol = 1e-3
    nsamp = int(sample_fraction * np.prod(dims))

    omega = sampling_operator(nsamp, dims)
    vec_data = get_sampling_vector(M, omega)
    mat_data = get_sampling_matrix(M, omega)
    save_image(mat_data, str(i+1)+'_'+str(sample_fraction)+'_'+str(rank)+'_pattern', 'results')
    
    print 'Image: '+str(rank)+'_pattern'+str(i+1)+'. Size: %dx%d. Rank: %d. Sample fraction: %.2f.' % (M.shape[0], 
                                                                                                       M.shape[1], 
                                                                                                       np.linalg.matrix_rank(M),
                                                                                                       sample_fraction)

    t0 = time()
    X_svt = get_completion(omega, vec_data, rank, dims, 'SVT', max_iter, tol)
    t1 = time()
    rmse_svt = RMSE(X_svt, M)
    rank_svt = np.linalg.matrix_rank(X_svt)
    X_svt /= np.max(X_svt)
    save_image(X_svt, str(i+1)+'_'+str(sample_fraction)+'_'+str(rank)+'_pattern_SVT_RMSE_%.5f' % rmse_svt, 'results')
    print 'SVT. Rank: %d. RMSE: %.5f. Time: %d min %d sec.' % (rank_svt, rmse_svt, int(t1-t0) // 60, int(t1-t0) % 60)

    t0 = time()
    X_svp = get_completion(omega, vec_data, rank, dims, 'SVP', max_iter, tol)
    t1 = time()
    rmse_svp = RMSE(X_svp, M)
    rank_svp = np.linalg.matrix_rank(X_svp)
    X_svp /= np.max(X_svp)
    save_image(X_svp, str(i+1)+'_'+str(sample_fraction)+'_'+str(rank)+'_pattern_SVP_RMSE_%.5f' % rmse_svp, 'results')
    print 'SVP. Rank: %d. RMSE: %.5f. Time: %d min %d sec.' % (rank_svp, rmse_svp, int(t1-t0) // 60, int(t1-t0) % 60)

    t0 = time()
    X_si = get_completion(omega, vec_data, rank, dims, 'SoftImpute', int(max_iter), tol)
    t1 = time()
    rmse_si = RMSE(X_si, M)
    rank_si = np.linalg.matrix_rank(X_si)
    X_si /= np.max(X_si)
    save_image(X_si, str(i+1)+'_'+str(sample_fraction)+'_'+str(rank)+'_pattern_SI_RMSE_%.5f' % rmse_si, 'results')
    print ' SI. Rank: %d. RMSE: %.5f. Time: %d min %d sec.' % (rank_si, rmse_si, int(t1-t0) // 60, int(t1-t0) % 60)
    print ''

Image: 11_pattern1. Size: 300x500. Rank: 11. Sample fraction: 0.10.
SVT. Rank: 45. RMSE: 0.10606. Time: 4 min 28 sec.
SVP. Rank: 11. RMSE: 0.00496. Time: 0 min 15 sec.
 SI. Rank: 300. RMSE: 0.07781. Time: 0 min 5 sec.

Image: 11_pattern2. Size: 1000x820. Rank: 11. Sample fraction: 0.10.
SVT. Rank: 11. RMSE: 0.00944. Time: 2 min 50 sec.
SVP. Rank: 11. RMSE: 0.00387. Time: 0 min 19 sec.
 SI. Rank: 820. RMSE: 0.05083. Time: 0 min 23 sec.

Image: 11_pattern3. Size: 1000x1000. Rank: 11. Sample fraction: 0.10.
SVT. Rank: 11. RMSE: 0.00140. Time: 3 min 26 sec.
SVP. Rank: 11. RMSE: 0.00175. Time: 0 min 11 sec.
 SI. Rank: 1000. RMSE: 0.05790. Time: 0 min 27 sec.

Image: 11_pattern4. Size: 1000x1000. Rank: 11. Sample fraction: 0.10.
SVT. Rank: 11. RMSE: 0.00204. Time: 3 min 48 sec.
SVP. Rank: 11. RMSE: 0.00116. Time: 0 min 14 sec.
 SI. Rank: 1000. RMSE: 0.03344. Time: 0 min 27 sec.

Image: 11_pattern5. Size: 600x600. Rank: 11. Sample fraction: 0.10.
SVT. Rank: 28. RMSE: 0.01244. Time: 4 min 40 s

In [ ]:
for i in xrange(5):
    rank = 7
    image = load_image('pattern'+str(i+1))
    save_image(image, 'small_pattern'+str(i+1), 'original')
    gray_image = get_gray_image(image)
    reduced_image = get_reduced_image(gray_image, rank)
    save_image(reduced_image, str(rank)+'_pattern'+str(i+1), 'low_rank')
    M = reduced_image.copy()
    sample_fraction = 0.05
    dims = M.shape
    max_iter = int(1e3)
    tol = 1e-3
    nsamp = int(sample_fraction * np.prod(dims))

    omega = sampling_operator(nsamp, dims)
    vec_data = get_sampling_vector(M, omega)
    mat_data = get_sampling_matrix(M, omega)
    save_image(mat_data, str(i+1)+'_'+str(sample_fraction)+'_'+str(rank)+'_pattern', 'results')
    
    print 'Image: '+str(rank)+'_pattern'+str(i+1)+'. Size: %dx%d. Rank: %d. Sample fraction: %.2f.' % (M.shape[0], 
                                                                                                       M.shape[1], 
                                                                                                       np.linalg.matrix_rank(M),
                                                                                                       sample_fraction)

    t0 = time()
    X_svt = get_completion(omega, vec_data, rank, dims, 'SVT', max_iter, tol)
    t1 = time()
    rmse_svt = RMSE(X_svt, M)
    rank_svt = np.linalg.matrix_rank(X_svt)
    X_svt /= np.max(X_svt)
    save_image(X_svt, str(i+1)+'_'+str(sample_fraction)+'_'+str(rank)+'_pattern_SVT_RMSE_%.5f' % rmse_svt, 'results')
    print 'SVT. Rank: %d. RMSE: %.5f. Time: %d min %d sec.' % (rank_svt, rmse_svt, int(t1-t0) // 60, int(t1-t0) % 60)

    t0 = time()
    X_svp = get_completion(omega, vec_data, rank, dims, 'SVP', max_iter, tol)
    t1 = time()
    rmse_svp = RMSE(X_svp, M)
    rank_svp = np.linalg.matrix_rank(X_svp)
    X_svp /= np.max(X_svp)
    save_image(X_svp, str(i+1)+'_'+str(sample_fraction)+'_'+str(rank)+'_pattern_SVP_RMSE_%.5f' % rmse_svp, 'results')
    print 'SVP. Rank: %d. RMSE: %.5f. Time: %d min %d sec.' % (rank_svp, rmse_svp, int(t1-t0) // 60, int(t1-t0) % 60)

    t0 = time()
    X_si = get_completion(omega, vec_data, rank, dims, 'SoftImpute', int(max_iter), tol)
    t1 = time()
    rmse_si = RMSE(X_si, M)
    rank_si = np.linalg.matrix_rank(X_si)
    X_si /= np.max(X_si)
    save_image(X_si, str(i+1)+'_'+str(sample_fraction)+'_'+str(rank)+'_pattern_SI_RMSE_%.5f' % rmse_si, 'results')
    print ' SI. Rank: %d. RMSE: %.5f. Time: %d min %d sec.' % (rank_si, rmse_si, int(t1-t0) // 60, int(t1-t0) % 60)
    print ''

Image: 7_pattern1. Size: 300x500. Rank: 7. Sample fraction: 0.05.


In [45]:
for i in xrange(5):
    rank = 11
    image = load_image('pattern'+str(i+1))
    save_image(image, 'small_pattern'+str(i+1), 'original')
    gray_image = get_gray_image(image)
    reduced_image = get_reduced_image(gray_image, rank)
    save_image(reduced_image, str(rank)+'_pattern'+str(i+1), 'low_rank')
    M = reduced_image.copy()
    sample_fraction = 0.05
    dims = M.shape
    max_iter = int(1e3)
    tol = 1e-3
    nsamp = int(sample_fraction * np.prod(dims))

    omega = sampling_operator(nsamp, dims)
    vec_data = get_sampling_vector(M, omega)
    mat_data = get_sampling_matrix(M, omega)
    save_image(mat_data, str(i+1)+'_'+str(sample_fraction)+'_'+str(rank)+'_pattern', 'results')
    
    print 'Image: '+str(rank)+'_pattern'+str(i+1)+'. Size: %dx%d. Rank: %d. Sample fraction: %.2f.' % (M.shape[0], 
                                                                                                       M.shape[1], 
                                                                                                       np.linalg.matrix_rank(M),
                                                                                                       sample_fraction)

    t0 = time()
    X_svt = get_completion(omega, vec_data, rank, dims, 'SVT', max_iter, tol)
    t1 = time()
    rmse_svt = RMSE(X_svt, M)
    rank_svt = np.linalg.matrix_rank(X_svt)
    X_svt /= np.max(X_svt)
    save_image(X_svt, str(i+1)+'_'+str(sample_fraction)+'_'+str(rank)+'_pattern_SVT_RMSE_%.5f' % rmse_svt, 'results')
    print 'SVT. Rank: %d. RMSE: %.5f. Time: %d min %d sec.' % (rank_svt, rmse_svt, int(t1-t0) // 60, int(t1-t0) % 60)

    t0 = time()
    X_svp = get_completion(omega, vec_data, rank, dims, 'SVP', max_iter, tol)
    t1 = time()
    rmse_svp = RMSE(X_svp, M)
    rank_svp = np.linalg.matrix_rank(X_svp)
    X_svp /= np.max(X_svp)
    save_image(X_svp, str(i+1)+'_'+str(sample_fraction)+'_'+str(rank)+'_pattern_SVP_RMSE_%.5f' % rmse_svp, 'results')
    print 'SVP. Rank: %d. RMSE: %.5f. Time: %d min %d sec.' % (rank_svp, rmse_svp, int(t1-t0) // 60, int(t1-t0) % 60)

    t0 = time()
    X_si = get_completion(omega, vec_data, rank, dims, 'SoftImpute', int(max_iter), tol)
    t1 = time()
    rmse_si = RMSE(X_si, M)
    rank_si = np.linalg.matrix_rank(X_si)
    X_si /= np.max(X_si)
    save_image(X_si, str(i+1)+'_'+str(sample_fraction)+'_'+str(rank)+'_pattern_SI_RMSE_%.5f' % rmse_si, 'results')
    print ' SI. Rank: %d. RMSE: %.5f. Time: %d min %d sec.' % (rank_si, rmse_si, int(t1-t0) // 60, int(t1-t0) % 60)
    print ''

Image: 11_pattern1. Size: 300x500. Rank: 11. Sample fraction: 0.05.


KeyboardInterrupt: 